In [1]:
import ipywidgets as ipw

FREQUENCIES = ( '1D', '1M', '3M', '6M', '12M' )
DCC         = ( 'Act/360', 'Act/365', 'Act/365F', '30/360' )

out = ipw.Output()

class IHaveWidget( object ):
    
    def _ipython_display_( self ):
        display( self.widget() )

class Leg( object ):
    
    def __init__( self ):
        self.payFreq = ipw.Dropdown( description='Freq', options = FREQUENCIES, value='3M' )
        self.dcc     = ipw.Dropdown( description='DCC', options = DCC, value='Act/360' )
        
    def values( self ):
        values = {}
        values.update( { 'payFreq' : self.payFreq.value,
                         'dcc'     : self.dcc.value } )
        return values
   
class FixedLeg( Leg, IHaveWidget ):
    
    def widget( self ):
        w = ipw.VBox( [ self.payFreq, 
                        self.dcc ] )
        return w
    

class FloatLeg( Leg ):
    def __init__( self ):
        super( FloatLeg, self ).__init__()
        self.rateIndex = ipw.Text( value = 'USD-LIBOR-3m', description='Rate Index' )
        self.resetAdj  = ipw.Text( value = '-2D(USD,GBP)', description='Reset Adj' )
        
    def widget( self ):
        w = ipw.VBox( [ self.payFreq, 
                        self.dcc, 
                        self.rateIndex, 
                        self.resetAdj ] )
        return w
    
    def values( self ):
        values = super( FloatLeg, self ).values()
        values.update( { 'rateIndex' : self.rateIndex.value,
                         'resetAdj'  : self.resetAdj.value } )
        return values
    
    
class NamedObject( object ):
    
    def __init__( self, name):
        self.name       = ipw.Text( value = name )
        
    def values( self ):
        values = { 'name' : self.name.value }
        return values
    
    
class Instrument( NamedObject ):
    
    def __init__( self, name ):
        super( Instrument, self ).__init__( name )
        self.instType  = ipw.Text( value = str( self.__class__.__name__ ) )
        
    def values( self ):
        values = super( Instrument, self ).values()
        values.update( { 'instType' : self.instType.value } )
        return values
    
    
class Swap( Instrument, IHaveWidget ):
    def __init__( self, name ):
        super( Swap, self ).__init__( name )
        self.fixedLeg = FixedLeg()
        self.floatLeg = FloatLeg()
        
    def widget( self ):
        tabs = ipw.Tab( [ l.widget() 
                           for l in ( self.fixedLeg, 
                                      self.floatLeg ) ] )
        tabs.set_title( 0, 'Fixed Leg' )
        tabs.set_title( 1, 'Floating Leg')
        w = ipw.VBox( [ self.name, tabs ] )
        return w
    
    def values( self ):
        values = super( Swap, self ).values()
        values.update( { 'fixedLeg' : self.fixedLeg.values(), 
                         'floatLeg' : self.floatLeg.values() } )
        return values

    
class InstrumentEditor( object ):
    
    def __init__( self, instrument ):
        self.instrument = instrument
        self._widget = self.instrument.widget()
        self._editor = ipw.HBox( [ self._widget ] )
        
    def _ipython_display_( self ):
        display( self._editor )



In [2]:
instrument = Swap( 'swap1' )
editor = InstrumentEditor( instrument )
editor

In [3]:
editor._editor.children = [Swap( 'swap2' ).widget() ]

In [4]:
def _create( b ):
    out.clear_output()
    with out:
        print( instrument.values() )
    
button = ipw.Button( description='Create swap')
button.on_click( _create )
display( button, out )

Button(description='Create swap', style=ButtonStyle())

Output()